In [1]:
import json
import hashlib
from flask import Flask, jsonify
import datetime
import os

In [2]:
class MainChain:
    def __init__(self):
        self.chain = []
        self.createCoin(proof = 1, previousProof = "0")
    
    def createCoin(self, proof, previousProof):
        coin = {
            "index": len(self.chain) + 1,
            "datetime": str(datetime.datetime.now()),
            "proof": proof,
            "previousProof": previousProof
        }
        self.chain.append(coin)
        return coin
    def getLastCoin(self):
        return self.chain[-1]
    
    def proofOfWork(self, previousProof):
        new_proof = 1
        solved = False
        while solved is False:
            hasProblem = hashlib.sha256(str(new_proof ** 2 - previousProof ** 2).encode()).hexdigest()
            if hasProblem[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    def hashBlock(self, coin):
        encodedCoin = json.dumps(coin, sort_keys=True).encode()
        return hashlib.sha256(encodedCoin).hexdigest()
    
    def isChainValid(self, chain):
        previous_block = chain[0]
        current_index = 1
        
        while current_index < len(chain):
            block = chain[current_index]
            prevHash = block["previousProof"]
            
            if prevHash != hashBlock(previous_block):
                return False
            currentPoof = block["proof"]
            prevProof = previous_block["proof"]
            hexProblem = hashlib.sha256(str(currentPoof **2 - prevProof ** 2).encode()).hexdigest()
            if hexProblem[:4] != "0000":
                return False
        return True

In [3]:
app = Flask(__name__)

In [4]:
mana = MainChain()

In [5]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousCoin = mana.getLastCoin()
    previousProof = previousCoin["proof"]
    proof = mana.proofOfWork(previousProof)
    prevHash = mana.hashBlock(previousCoin)
    coin = mana.createCoin(proof, prevHash)
    response = {
        "success": "you have successfully mined a Mana coin",
        "index": coin["index"],
        "proof": coin["proof"],
        "previousHash": coin["previousProof"],
        "timestamp": coin["datetime"]
    }
    return jsonify(response), 200

In [6]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": mana.chain,
        "length": len(mana.chain)
    }
    return jsonify(response), 200

In [7]:
if __name__=="__main__":
    app.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4444)))

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:4444/ (Press CTRL+C to quit)
